<a href="https://colab.research.google.com/github/mozhgans/Machine-Learning-Algorithms/blob/main/Tensorflow_Beginner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This short introduction uses Keras to:

Load a prebuilt dataset.

Build a neural network machine learning model that classifies images.

Train this neural network.

Evaluate the accuracy of the model.

Set up TensorFlow

Import TensorFlow into your program to get started:

In [ ]:
import tensorflow as tf
print("tensorflow version:", tf.__version__)

tensorflow version: 2.7.0


Load a dataset:

Load and prepare the MNIST dataset. Convert the sample data from integers to floating-point numbers:

In [ ]:
mnist = tf.keras.datasets.mnist
mnist

<module 'keras.api._v2.keras.datasets.mnist' from '/usr/local/lib/python3.7/dist-packages/keras/api/_v2/keras/datasets/mnist/__init__.py'>

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


Build a machine learning model:

Build a tf.keras.Sequential model by stacking layers.

When to use a Sequential model:

A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.

https://www.tensorflow.org/guide/keras/sequential_model

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example, the model returns a vector of logits or log-odds scores, one for each class.

In [ ]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.13848424,  0.07517522,  0.00898615,  0.70145357,  0.07138107,
         0.08409797,  0.12883909,  0.289892  , -0.32842115,  0.45452484]],
      dtype=float32)

The tf.nn.softmax function converts these logits to probabilities for each class:

In [ ]:
tf.nn.softmax(predictions).numpy()

array([[0.07313258, 0.09055283, 0.08475327, 0.16939136, 0.09020991,
        0.09136443, 0.095545  , 0.11224121, 0.06048147, 0.13232791]],
      dtype=float32)

Note: It is possible to bake the tf.nn.softmax function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

Define a loss function for training using losses.SparseCategoricalCrossentropy, which takes a vector of logits and a True index and returns a scalar loss for each example.

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.
This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.math.log(1/10) ~= 2.3.

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

2.392899

Before you start training, configure and compile the model using Keras Model.compile. Set the optimizer class to adam, set the loss to the loss_fn function you defined earlier, and specify a metric to be evaluated for the model by setting the metrics parameter to accuracy.

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

Train and evaluate your model:

Use the Model.fit method to adjust your model parameters and minimize the loss:

In [ ]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2956 - accuracy: 0.9139
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1427 - accuracy: 0.9571
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1073 - accuracy: 0.9675
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0866 - accuracy: 0.9731
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0739 - accuracy: 0.9765


The Model.evaluate method checks the models performance, usually on a "Validation-set" or "Test-set".

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0728 - accuracy: 0.9764 - 498ms/epoch - 2ms/step


[0.07283315062522888, 0.9764000177383423]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the TensorFlow tutorials.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[3.26454324e-06, 5.99432681e-08, 4.63232618e-06, 9.51350012e-05,
        3.00897363e-11, 1.65395377e-07, 5.22938789e-13, 9.99870539e-01,
        1.33047424e-05, 1.29708815e-05],
       [2.92222154e-07, 1.16908423e-05, 9.99883652e-01, 1.01976395e-04,
        5.77697356e-18, 1.10918677e-06, 4.44721806e-08, 8.94955442e-13,
        1.36078324e-06, 1.23418940e-12],
       [1.62418416e-07, 9.99318361e-01, 1.15955072e-05, 9.65064555e-06,
        1.01069827e-05, 3.41020404e-06, 6.15419958e-06, 5.39682340e-04,
        1.00741447e-04, 1.71555911e-07],
       [9.99918938e-01, 2.95898617e-09, 7.92908759e-05, 3.24559366e-08,
        2.06955830e-09, 2.72191913e-09, 1.12705675e-06, 2.59299441e-07,
        9.41596356e-10, 3.51951428e-07],
       [2.87269504e-05, 2.00488270e-09, 9.11606840e-05, 1.31878878e-07,
        9.91838634e-01, 1.78377675e-07, 2.59946842e-06, 3.05520440e-03,
        1.63258039e-06, 4.98180836e-03]], dtype=float32)>

Conclusion:

Congratulations! You have trained a machine learning model using a prebuilt dataset using the Keras API.
For more examples of using Keras, check out the tutorials. To learn more about building models with Keras, read the guides. If you want learn more about loading and preparing data, see the tutorials on image data loading or CSV data loading.